In [8]:
import pandas as pd
import numpy as np
import random
import string

# generate numbers
nums = np.random.randint(0, high=100, size=(100000,))

attrs = []
for n in nums:
    s = ''.join([random.choice(string.ascii_letters) for n in range(5)])
    attrs.append(str(n) + ' ' + s)




['69 FlBWH', '11 bPEfH', '89 dayrm', '40 neIMl', '91 bCPJA', '71 LuOee', '56 QJMTD', '53 Iixpn', '88 JVCAV', '81 JJuLa', '44 KgqXT', '66 lYAHR', '78 Uilym', '37 aZLnw', '91 euzQK', '83 pMvzh', '99 anFDw', '11 FEYog', '13 IQQfa', '71 GBwuf', '23 Qdusx', '1 IDAyG', '34 nCIdP', '15 khXSN', '77 gNEIG', '75 IaHsU', '67 wHihZ', '87 tvVcO', '24 mZThF', '95 OBkVt', '85 UvNcA', '55 szGKA', '33 sWInJ', '42 mKURs', '5 lrNqb', '27 cASrU', '91 KnzxG', '75 bbwIJ', '30 FhgJy', '21 dJwUY', '87 ezFGF', '6 eXOit', '7 HYdzT', '66 EfgsN', '71 TfhxS', '55 mvEsT', '43 jUzOh', '34 ZNJsn', '37 VCRkZ', '45 cRehA', '62 zzepA', '28 xsKGH', '56 CCDpI', '55 VPGaV', '27 XPqdd', '99 arZlv', '26 QGrGa', '80 tsNrW', '39 HRItz', '13 aabZR', '82 ltIKz', '10 Pwdoy', '51 tBccD', '87 Ntenb', '2 WfAcm', '76 UYWTn', '17 HtyXJ', '8 lwQBt', '31 fangQ', '81 kSdvE', '25 CELYr', '70 VNDqe', '54 GvuOR', '70 Trply', '29 bnvgc', '31 jfEoY', '62 uOcwv', '90 HYyUo', '78 RfixO', '33 eZuey', '13 swqqw', '93 WzgCI', '71 AzROL', '41 ziAwf

In [12]:
train_nums, val_nums, test_nums = nums[:80000], nums[80000:90000], nums[90000:]
train_attrs, val_attrs, test_attrs = attrs[:80000], attrs[80000:90000], attrs[90000:]

In [14]:
train_df = pd.DataFrame({"Numerical_value":train_nums,"Attribute_value":train_attrs})
train_df.to_csv('data/simple_train.csv')

val_df = pd.DataFrame({"Numerical_value":val_nums,"Attribute_value":val_attrs})
val_df.to_csv('data/simple_val.csv')

test_df = pd.DataFrame({"Numerical_value":test_nums,"Attribute_value":test_attrs})
test_df.to_csv('data/simple_test.csv')

In [20]:
from dataloading import *
from model import *
from utils import *

data_dir = "data"
filename="simple_train.csv"
inputs,targets = read_data(data_dir,None,filename)

# chunk
BATCH_SIZE=512
n_chunks = int(math.ceil(1.0*inputs.size()[1]/BATCH_SIZE))
inputs = torch.chunk(inputs, n_chunks, dim=1) 
targets = torch.chunk(targets, n_chunks, dim=1) 




AttributeError: 'tuple' object has no attribute 'size'

In [22]:
data_dir = "data"
filename="simple_val.csv"
val_inputs,val_targets = read_data(data_dir,None,filename)

# chunk
BATCH_SIZE=512
n_chunks = int(math.ceil(1.0*val_inputs.size()[1]/BATCH_SIZE))
val_inputs = torch.chunk(val_inputs, n_chunks, dim=1) 
val_targets = torch.chunk(val_targets, n_chunks, dim=1) 

In [19]:
computing_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_digits =10
n_chars=256

config = {
            'epochs':100,
            'N_early_stop':30,
            'batch_size':512,
            'learning_rate':0.001,
            'weight_decay':0,
            'teacher_forcing_ratio':1.0,
            'hidden_dim':512,
            'n_layers':2, 
            'enc': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },
            'dec': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },

            'input_dim':n_chars+4,
            'output_dim':n_digits+5,

            'verbose':True
        }

    
model = init_seq2seq(config, computing_device)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])
criterion = nn.CrossEntropyLoss(ignore_index=output_pad_index)

In [21]:
def train_and_validate_2(output_dir,config,test_type, train_inputs, train_targets, val_inputs, val_targets, computing_device, N=5):
    #print(output_dir)
    #output_dir = os.path.join('output',output_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_file = 'bs={}_lr={}_wd={}_tf={}_hd={}_id={}_fold={}'.format(config['batch_size'],config['learning_rate'],config['weight_decay'],config['teacher_forcing_ratio'],config['enc']['hid_dropout'],config['enc']['input_dropout'],test_type)
    #print(output_file)
    output_filepath = os.path.join(output_dir,output_file+'.csv')   
    
    model = init_seq2seq(config, computing_device)
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])
    criterion = nn.CrossEntropyLoss(ignore_index=output_pad_index)

    verbose=config['verbose']
    #train_inputs, train_targets, val_inputs, val_targets = split_data(filenames_by_type,test_type, BATCH_SIZE=config['batch_size'])

    avg_val_loss=0.0
    avg_train_loss=0.0
    min_val_loss=100
    min_epoch=0
    best_state_dict=None
    total_start=time.time()
    for epoch in range(config['epochs']):
        # train 
        if verbose:
            print('...training')
        start=time.time()
        train_loss = train(model, train_inputs, train_targets, optimizer, criterion, computing_device,config)
        if verbose:
            print('   epoch {}: train_loss:{}, time:{}'.format(epoch,train_loss,time.time()-start))

        #validate
        if verbose:
            print('...validating')
        start=time.time()
        val_loss = validate(model, val_inputs, val_targets, optimizer, criterion, computing_device)
        if verbose:
            print('   epoch {}: val_loss:{}, time:{}'.format(epoch,val_loss,time.time()-start))
    
        avg_val_loss+=val_loss
        avg_train_loss+=train_loss
        
        if epoch%N==0 and epoch!=0:
            avg_val_loss/=N
            avg_train_loss/=N
            
            with open(output_filepath, 'a') as file: 
                file.write('{},{},{},{}\n'.format(epoch,avg_val_loss,avg_train_loss,time.time()-total_start))
            # update min, state_dict
            if avg_val_loss<min_val_loss:
                min_val_loss=avg_val_loss
                min_epoch=epoch 
                best_state_dict = model.state_dict()
            # if not decreasing for a while
            elif epoch - min_epoch >= config['N_early_stop']:
                with open(os.path.join(output_dir,'output_val_loss.txt'), 'a') as file: 
                    file.write('{},{},{}\n'.format(output_file,min_epoch,min_val_loss))
                if best_state_dict:
                    PATH = os.path.join(output_dir,output_file+'_best.pt') 
                    torch.save(best_state_dict, PATH)
                return min_val_loss, min_epoch, config
            avg_val_loss=0.0
            avg_train_loss=0.0
            total_start=time.time()
            
    with open(os.path.join(output_dir,'output_val_loss.txt'), 'a') as file: 
        file.write('{},{},{}\n'.format(output_file,min_epoch,min_val_loss))
    if best_state_dict:
        PATH = os.path.join(output_dir,output_file+'_best.pt') 
        torch.save(best_state_dict, PATH)
    else:
        PATH = os.path.join(output_dir,output_file+'_final.pt') 
        torch.save(model.state_dict(), PATH)
    
    return min_val_loss, min_epoch, config

In [ ]:
output_dir='SIMPLE_hd={}_nl={}'.format(config['hidden_dim'],config['n_layers'])
output_dir = os.path.join('output',output_dir)


test_type='E'
train_and_validate_2(output_dir,config,test_type, inputs, targets, val_inputs, val_targets, computing_device, N=5)

...training
